## The new `BosonicOp` and `BosonicLinearMapper` in Qiskit Nature

Francesco Troisi
Qiskit Demo Day - June 15th, 2023

Overview: See [related issue](https://github.com/qiskit-community/qiskit-nature/issues/958) and [related PR](https://github.com/qiskit-community/qiskit-nature/pull/1085)

## Bosonic Operator

A Bosonic operator allows to create and annihilitate a Boson, which is a particle with integer spin (like photon and phonons).
The behavior of this class is very similar to the `FermionicOp`, so if one is familiar with that it is going to be straightforward to use this. One has to remember that the BosonicOp follows the commutation relations (instead of the anti-commutation)

In [21]:
from qiskit_nature.second_q.operators import BosonicOp, FermionicOp

In [12]:
# Create a creation operator
bosonic_op = BosonicOp({'+_0': 1}, num_modes=1)
# Create an annihilation operator
bosonic_op = BosonicOp({'-_0': 1}, num_modes=1)

The BosonicOp class supports all functionalities supported by FermionicOp

In [13]:
bosonic_op1 = BosonicOp({'+_0 -_0 -_1 +_1': 2}, num_modes=2)
bosonic_op2 = BosonicOp({'+_2 -_2': 1.5, '+_0': 3}, num_modes=3)
# Sum
print("Sum")
print(bosonic_op1 + bosonic_op2)
# Multiplication
print("\nMultiplication")
print(0.5 * bosonic_op1 @ bosonic_op2)

Sum
Bosonic Operator
number modes=3, number terms=3
  2 * ( +_0 -_0 -_1 +_1 )
+ 1.5 * ( +_2 -_2 )
+ 3 * ( +_0 )

Multiplication
Bosonic Operator
number modes=3, number terms=2
  1.5 * ( +_0 -_0 -_1 +_1 +_2 -_2 )
+ 3.0 * ( +_0 -_0 -_1 +_1 +_0 )


Simplificiation and ordering operations

In [14]:
bosonic_op = BosonicOp({'+_0 -_0 -_1 +_0': 1}, num_modes=2)
# Normal order (reorder all operators such that all creation op are in front). Due to the commutation relations, when inverting -_0 +_0 we get 1 + +_0 -_0, which explains the two terms
print("Normal Ordering")
print(bosonic_op.normal_order())
# Index order (reorder all operators such that the lowest indices come first)
print("\nIndex Ordering")
print(bosonic_op.index_order())

Normal Ordering
Bosonic Operator
number modes=2, number terms=2
  1 * ( +_0 -_1 )
+ 1 * ( +_0 +_0 -_0 -_1 )

Index Ordering
Bosonic Operator
number modes=2, number terms=1
  (1+0j) * ( +_0 -_0 +_0 -_1 )


In [22]:
# Simplify
print("Simplify")
# Simplify can be used to reduce numerical noise:
print(BosonicOp({'+_0 -_0': 1}) + BosonicOp({'+_0 -_0': 1e-10}).simplify(atol=1e-9))

Simplify
Bosonic Operator
number modes=1, number terms=1
  1 * ( +_0 -_0 )


An important difference with FermionicOp, whose cause lies in the nature of particles, is that for bosons we can have apply the creation or annihilation operator multiple times

In [16]:
print("\nSimplify with two creation operators")
print("\nBosons:")
print(BosonicOp({'+_0 -_0 -_1 +_0 +_0': 1}, num_modes=2).simplify())
print("\nFermions:")
print(FermionicOp({'+_0 -_0 -_1 +_0 +_0': 1}, num_spin_orbitals=2).simplify())


Simplify with two creation operators

Bosons:
Bosonic Operator
number modes=2, number terms=1
  1 * ( +_0 -_0 -_1 +_0 +_0 )

Fermions:
Fermionic Operator
number spin orbitals=2, number terms=0
  


In [17]:
FermionicOp({'+_0 -_0 -_1 +_0': 1}, num_spin_orbitals=2).simplify()

FermionicOp({'+_0 -_1': (-1+0j)}, num_spin_orbitals=2, )

# Bosonic Linear Mapper

In order to use a the Bosonic operator, it is necessary to be able to map it to the Qubit space.
The Bosonic Linear Mapper class implements a mapping described in Section II.C of [A. Miessen et al., Quantum algorithms for quantum dynamics: A performance study on the spin-boson model](https://link.aps.org/doi/10.1103/PhysRevResearch.3.043212).
Note that it is compulsory to **set use_pauli_sum_op to False**, as this feature is now deprecated

The implemented formula is the following:
$$\hat{b}_k^\dagger \rightarrow \sum_{n_k = 0}^{n_k^{max} - 1} \sqrt{n_k + 1} \sigma_{n_k}^{+} \sigma_{n_k + 1}^{-}$$
$$\hat{b}_{k} \rightarrow \sum_{n_k = 0}^{n_k^{max} - 1} \sqrt{n_k + 1} \sigma_{n_k}^- \sigma_{n_k + 1}^+$$
Where
$\sigma_{n_k}^{\pm} = \frac{1}{2} \left(\sigma_{n_k}^x \pm i\sigma_{n_k}^y\right)$

In [18]:
from qiskit_nature.second_q.mappers import BosonicLinearMapper
from qiskit_nature import settings
settings.use_pauli_sum_op = False

In order to use it, one must initialize it and then call the map method

In [19]:
mapper_occ1 = BosonicLinearMapper(max_occupation=1)
boson1 = mapper_occ1.map(BosonicOp({'+_0 -_0': 1}))
print(boson1)

SparsePauliOp(['II', 'ZZ', 'ZI', 'IZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j])


The parameter `max_occupation` defines what is the state at which the maximum excitation can take place. This parameter has a huge impact on the size of the calculation (in terms of number of qubits required) and on the representation of the state.
In fact, one has to keep in mind that this mapper is not exact (alike the fermions, we would need an infinite number of qubits to represent a boson). As such, the higher the `max_occupation`, the better the state is represented

In [20]:
print("Max occupation = 1")
boson_p1 = mapper_occ1.map(BosonicOp({'+_0': 1}))
boson_m1 = mapper_occ1.map(BosonicOp({'-_0': 1}))
print(boson_p1)

print("\nMax occupation = 2")
mapper_occ2 = BosonicLinearMapper(max_occupation=2)
boson_p2 = mapper_occ2.map(BosonicOp({'+_0': 1}))
boson_m2 = mapper_occ2.map(BosonicOp({'-_0': 1}))
print(boson_p2)

print("\nMax occupation = 3")
mapper_occ3 = BosonicLinearMapper(max_occupation=3)
boson_p3 = mapper_occ3.map(BosonicOp({'+_0': 1}))
boson_m3 = mapper_occ3.map(BosonicOp({'-_0': 1}))
print(boson_p3)

Max occupation = 1
SparsePauliOp(['XX', 'YY', 'YX', 'XY'],
              coeffs=[0.25+0.j  , 0.25+0.j  , 0.  -0.25j, 0.  +0.25j])

Max occupation = 2
SparsePauliOp(['IXX', 'IYY', 'IYX', 'IXY', 'XXI', 'YYI', 'YXI', 'XYI'],
              coeffs=[0.25      +0.j        , 0.25      +0.j        , 0.        -0.25j      ,
 0.        +0.25j      , 0.35355339+0.j        , 0.35355339+0.j        ,
 0.        -0.35355339j, 0.        +0.35355339j])

Max occupation = 3
SparsePauliOp(['IIXX', 'IIYY', 'IIYX', 'IIXY', 'IXXI', 'IYYI', 'IYXI', 'IXYI', 'XXII', 'YYII', 'YXII', 'XYII'],
              coeffs=[0.25      +0.j        , 0.25      +0.j        , 0.        -0.25j      ,
 0.        +0.25j      , 0.35355339+0.j        , 0.35355339+0.j        ,
 0.        -0.35355339j, 0.        +0.35355339j, 0.4330127 +0.j        ,
 0.4330127 +0.j        , 0.        -0.4330127j , 0.        +0.4330127j ])
